# Smart signatures
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [5]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Check Purestake API

In [6]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19807070


## Using OR in a sensable way
* Either the payment is made to Bob (any amount)
* Or it is maximal 1 Algo (to anybody)

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [20]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

escrow_condition_1 =  (
    Txn.receiver() == Addr(Bob["public"])           # Receipient must be Bob
)

escrow_condition_2 =  (
    Txn.amount() <= max_amount                       # Requested amount must be smaller than max_amount
)

# Fancy fee condtion
fee_condition =  (Int(1000) * Txn.fee() <= Int(1000)*Int(1000) + Txn.amount() )  # Max fee 

safety_cond = And(
    Global.group_size() == Int(1),                   # Not part of a TX group
    Txn.type_enum() == TxnType.Payment,
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address()
    )

modesty2_pyteal = And(
    fee_condition,
    saftey_condition,
    Or(escrow_condition_1,escrow_condition_2)
)

##### Step 2: Compile PyTeal -> Teal

In [21]:
modesty2_teal = compileTeal(modesty2_pyteal, Mode.Signature, version=3)
print(modesty2_teal)

#pragma version 3
int 1000
txn Fee
*
int 1000
int 1000
*
txn Amount
+
<=
txn TypeEnum
int pay
==
global GroupSize
int 1
==
&&
txn RekeyTo
global ZeroAddress
==
&&
&&
txn Receiver
addr O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
==
txn Amount
int 1000000
<=
||
&&


##### Step 3: Compile Teal -> Bytecode for AVM

In [10]:
Modesty2 = algod_client.compile(modesty2_teal)
Modesty2

{'hash': 'VVGQ77BZX2VFBTNHCX42TVPZS7PHXWVIJ4WG2MB3C2DNQK37XU3JY5RD7A',
 'result': 'AyABwIQ9JgEgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYxBygSMQgiDhE='}

##### Step 4: Alice funds and deploys the smart signature

In [11]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(5.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modesty2['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19807080.
Waiting for round 19807080 to finish.
Waiting for round 19807081 to finish.
Transaction ESHLGUQDRFUF3VOLVCV7T3I4VPARZ6I6AAYOTUS3MIQB6OWDYM6Q confirmed in round 19807082.


##### Step 5: Alice informs Bob

In [12]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modesty2['result'])
print("Address of smart signature: ", Modesty2['hash'])

Alice communicates to Bob the following
Compiled smart signature: AyABwIQ9JgEgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYxBygSMQgiDhE=
Address of smart signature:  VVGQ77BZX2VFBTNHCX42TVPZS7PHXWVIJ4WG2MB3C2DNQK37XU3JY5RD7A


#### Step 6: Charlie makes a small request

In [13]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e6)              # <---------- OK!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Charlie['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19807091.
Waiting for round 19807091 to finish.
Waiting for round 19807092 to finish.
Transaction Q2GZM6V7RZG7CO3SIPSE2QIAWLFO6TDYKJHAXTN77AOP6I4LLUNQ confirmed in round 19807093.


#### Step 6: Dina makes a large request

In [14]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- too much for Dina!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Dina['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)


# Step 3: send
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)               # print entire error message

# Step 4: wait for confirmation
# No need to wait for confirmation
#txinfo = wait_for_confirmation(algod_client, txid)

transaction {_struct:{} Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Lsig:{_struct:{} Logic:[3 32 1 192 132 61 38 1 32 118 164 184 189 92 71 37 106 56 88 49 148 115 208 69 74 75 215 68 229 95 253 135 115 101 196 195 243 209 223 1 198 49 7 40 18 49 8 34 14 17] Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Args:[]} Txn:{_struct:{} Type:pay Header:{_struct:{} Sender:VVGQ77BZX2VFBTNHCX42TVPZS7PHXWVIJ4WG2MB3C2DNQK37XU3JY5RD7A Fee:{Raw:1000} FirstValid:19807097 LastValid:19808097 Note:[] GenesisID:testnet-v1.0 GenesisHash:JBR3KGFEWPEE5SAQ6IWU6EEBZMHXD4CZU6WCBXWGF57XBZIJHIRA Group:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA Lease:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] RekeyTo:AAAAAAA

#### Step 6: Bob makes a large request
* This is OK, because it is Bob!

In [15]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- ok for Bob!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)


# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19807101.
Waiting for round 19807101 to finish.
Waiting for round 19807102 to finish.
Transaction EGEJ4IZ2L6ARG77YNMBJUU65Z2BHWBD3SYTQMOFYW7JVH56BD7KA confirmed in round 19807103.
